In [1]:
# default_exp core.logging

# babilim.core.logging

> Logging for tensorflow and pytorch.

This code is under the MIT License.

In [2]:
# export
# MIT License
#
# Copyright (c) 2019 Michael Fuerst
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import datetime
import filecmp
import json
import os
import shutil
import time
from typing import Callable, List, Union

import time as __time
import datetime as __datetime

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

In [3]:
#export
__ailab_logfile = None
__logfile = None
__logfolder = None
__log_buffer = []
__last_progress = 0
__last_update = time.time()


DEBUG_VERBOSITY = False
PRINT_STATUS = True
PRINT_INFO = True
PRINT_WARN = True
PRINT_ERROR = True

PYTHON_IGNORE_LIST = ["__pycache__", "*.pyc", ".ipynb_checkpoints", "checkpoints", "logs", "dist", "docs", "*.egg-info",
                      "tfrecords", "*.code-workspace", ".git"]

In [4]:
#export
def format_time(t):
    hours, remainder = divmod(t, 3600)
    minutes, seconds = divmod(remainder, 60)
    return '%d:%02d:%02d' % (hours, minutes, seconds)

In [5]:
#export
def _ignore(candidate: str, forbidden_list: List[str]) -> bool:
    # Parse list to find simple placeholder notations
    start_list = []
    end_list = []
    for item in forbidden_list:
        if item.startswith("*"):
            end_list.append(item.replace("*", ""))
        if item.endswith("*"):
            start_list.append(item.replace("*", ""))
    # Test
    res = candidate in forbidden_list
    for item in start_list:
        res |= candidate.startswith(item)
    for item in end_list:
        res |= candidate.endswith(item)
    return res

In [6]:
#export
def __get_all_files(root: str, forbidden_list: List[str]) -> List[str]:
    all_files = []
    root_with_sep = root + os.sep
    for path, subdirs, files in os.walk(root):
        files = [x for x in files if not _ignore(x, forbidden_list)]
        subdirs[:] = [x for x in subdirs if not x.startswith(".") and not _ignore(x, forbidden_list)]
        for name in files:
            all_files.append(os.path.join(path, name).replace(root_with_sep, ""))
    return all_files

In [7]:
#export
def _get_loaded_files(root: str = None, forbidden_list: List[str] = PYTHON_IGNORE_LIST) -> List[str]:
    """
    Get a list of all files that correspond to loaded modules in the root folder.

    If root is None the current cwd is used.
    """
    if root is None:
        root = os.getcwd()

    cwd_files = __get_all_files(root, forbidden_list)
    # TODO filter out all files that are not loaded.

    return cwd_files

In [8]:
#export
def _get_backup_path(fname: str, outp_dir: str = None) -> str:
    assert outp_dir is not None

    return os.path.join(os.path.normpath(outp_dir), fname)

In [9]:
#export
def _copyfile(src: str, dst: str, follow_symlinks: bool = True, create_missing_dirs: bool = True) -> None:
    dst_dir = os.path.dirname(dst)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    shutil.copyfile(src, dst, follow_symlinks=follow_symlinks)

In [10]:
#export
def _write_log(*, obj: object) -> None:
    """
    Write a log to the logfile or console if none is available.
    Furthermore send it to the online server if it is connected.

    :param obj: The json serializable object to log.
    """
    global __ailab_logfile
    out_str = json.dumps(obj)
    if __ailab_logfile is None:
        warn("You should setup logger before using it. Call babilim.logger.set_logger(...).")
        warn(out_str)
    else:
        with open(__ailab_logfile, "a") as f:
            f.write(out_str + "\n")

In [11]:
#export
def log_value(*, name: str, value: object) -> None:
    """
    Log a value to the file or online server.
    :param name: The name of the value to be logged.
    :param value: The actual value. It can be anything that is json serializable.
    """
    date = {
        "timestamp": "{}".format(datetime.datetime.now()),
        "name": name,
        "value": value
    }
    _write_log(obj=date)

In [12]:
#export
def log_progress(goal: str, progress: float, score: float) -> None:
    """
    Update the progress value. Automatically also computes the ETA and updates it in the logs.
    :param goal: The current goal it tries to reach. ("training", "validating", "pause", "waiting", "done")
    :param progress: A value between 0 and 1 indicating the progress, where 1 means done. The value should grow monotonic.
    :param score: The score that the approach achieved.
    """
    global __last_progress
    global __last_update

    assert 0 <= progress <= 1, progress

    delta_t = time.time() - __last_update
    delta_p = max(progress - __last_progress, 1E-6)
    __last_update = time.time()
    __last_progress = progress
    eta = (1.0 - progress) / delta_p * delta_t
    if goal == "pause" or goal == "waiting":
        eta = 0

    date = {
        "timestamp": "{}".format(datetime.datetime.now()),
        "eta": format_time(eta),
        "goal": str(goal),
        "score": int(score * 1000) / 1000,
        "progress": int(progress * 1000) / 1000
    }
    _write_log(obj=date)

In [13]:
#export
def _log_code(*, log_folder: str, forbidden_list: list = []) -> None:
    """
    Log the code of the current working directory into the src folder of your checkpoint path.

    :param chkpt_path: The checkpoint folder.
    :param forbidden_list: The list of the forbidden files.
    """
    outp_dir = os.path.join(log_folder, "src")
    forbidden_list.extend(PYTHON_IGNORE_LIST)

    loaded_files = _get_loaded_files(forbidden_list=forbidden_list)
    # Copy preparation code to output location and load the module.
    for f in loaded_files:
        f_backup = _get_backup_path(f, outp_dir=outp_dir)
        _copyfile(f, f_backup)

In [14]:
#export
def _is_code_log_up_to_date(*, log_folder: str, forbidden_list: list = []) -> bool:
    """
    Check if the code in the logs is up to date or needs updates.

    :param chkpt_path: The checkpoint folder.
    :param forbidden_list: The list of the forbidden files.
    :return: True if the files are up to date, False if not.
    """
    outp_dir = os.path.join(log_folder, "src")
    if not os.path.exists(outp_dir):
        return False
    forbidden_list.extend(PYTHON_IGNORE_LIST)
    loaded_files = _get_loaded_files(forbidden_list=forbidden_list)

    for f in loaded_files:
        f_backup = _get_backup_path(f, outp_dir=outp_dir)
        # Check if data is already up to date
        if not os.path.exists(f_backup) or not filecmp.cmp(f, f_backup):
            return False

    return True

In [15]:
#export
def log_image(*, name: str, data: np.ndarray = None) -> None:
    """
    Log an image.
    :param name: The name of the image (e.g. "example.png").
    :param data: The data (optional) if none is provided it is assumed that a pyplot figure should be saved.
    """
    global __logfolder
    if __logfolder is None:
        warn("Cannot log images when logger is not setup. Call logger.setup first")
        return
    fname = os.path.join(__logfolder, "images", name)
    image_folder = os.path.join(*(fname.split("/")[:-1]))
    if not os.path.exists(image_folder):
        os.makedirs(image_folder)
    if data is None:
        plt.savefig(fname)
    else:
        scipy.misc.imsave(fname, data)

In [16]:
#export
def close(reason: str = None) -> None:
    """
    Close the logger for a given reason.
    
    If none is provided there is no final progress written. Provide a reason, if you do not manually set the final progress before.
    A training loop typically manually sets the progress, so you will not need a reason in that case.
    
    :param reason: The reason for the closing of the logger. It is recommended to use "done", "paused", "failed" as reason.
    """
    global __ailab_logfile
    global __last_progress
    global __last_update
    if __ailab_logfile is None:
        raise RuntimeError("You must setup the logger before you can close it!")
    if reason is not None:
        log_progress(goal=reason, progress=1.0, score=0.0)

    __ailab_logfile = None
    __last_progress = 0
    __last_update = time.time()

In [17]:
#export
def set_logger(log_file: str) -> None:
    """
    Setup the logger.
    
    Creates the log folder, a src folder inside the log folder where it copies the current working directory and creates a log.txt, for logging progress.

    :param log_file: File where the warns, errors, etc. should be written. The folder in which the log file is is used for checkpointing.
    """
    global __ailab_logfile
    global __logfile
    global __logfolder
    global __log_buffer
    if __ailab_logfile is not None:
        raise RuntimeError("You must not setup logger twice!")

    log_folder = os.path.join(*(log_file.split("/")[:-1]))
    __logfolder = log_folder
    if not os.path.exists(log_folder):
        os.makedirs(log_folder)

    if not _is_code_log_up_to_date(log_folder=log_folder):
        _log_code(log_folder=log_folder)
    __ailab_logfile = os.path.join(log_folder, "ailab.log")
    log_progress(goal="waiting", progress=0, score=0)

    __logfile = log_file
    with open(__logfile, "a") as f:
        for data in __log_buffer:
            f.write(data + "\n")
        __log_buffer = []

In [ ]:
#export
def get_log_path() -> Union[str, None]:
    global __logfolder
    return __logfolder

In [18]:
#export
def get_timestamp() -> str:
    """
    Create a string for the current timestamp.
    
    :return: Current date and time as a string suitable for a logfolder filename.
    """
    time_stamp = __datetime.datetime.fromtimestamp(__time.time()).strftime('%Y-%m-%d_%H.%M.%S')
    return time_stamp

In [19]:
#export
def create_checkpoint_structure() -> str:
    """
    Create a checkpoint structure in the log folder.
    """
    if not os.path.exists(os.path.join(__logfolder, "train")):
        os.makedirs(os.path.join(__logfolder, "train"))
    if not os.path.exists(os.path.join(__logfolder, "dev")):
        os.makedirs(os.path.join(__logfolder, "dev"))
    if not os.path.exists(os.path.join(__logfolder, "test")):
        os.makedirs(os.path.join(__logfolder, "test"))
    if not os.path.exists(os.path.join(__logfolder, "checkpoints")):
        os.makedirs(os.path.join(__logfolder, "checkpoints"))
    if not os.path.exists(os.path.join(__logfolder, "images")):
        os.makedirs(os.path.join(__logfolder, "images"))

In [20]:
def status(msg: str, end: str = "\n") -> None:
    """
    Print something with a timestamp.
    Useful for logging.
    Babilim internally uses this for all its log messages.

    .. code-block:: python

        from babilim import tprint
        tprint("This is a log message.")
        # [yyyy-mm-dd HH:MM:SS] This is a log message.

    :param msg: The message to print.
    :param end: The line ending. Defaults to "\n" but can be set to "" to not have a linebreak.
    """
    if PRINT_STATUS:
        time_stamp = __datetime.datetime.fromtimestamp(__time.time()).strftime('%Y-%m-%d %H:%M:%S')
        data = "[{}] STAT {}".format(time_stamp, msg)
        print("\r{}".format(data), end=end)
        if end != "":
            if __logfile is not None:
                with open(__logfile, "a") as f:
                    f.write(data + "\n")
            else:
                __log_buffer.append(data)

In [21]:
#export
def info(msg: str, end: str= "\n") -> None:
    """
    Print something with a timestamp.
    Useful for logging.
    Babilim internally uses this for all its log messages.

    .. code-block:: python

        from babilim import tprint
        tprint("This is a log message.")
        # [yyyy-mm-dd HH:MM:SS] This is a log message.

    :param msg: The message to print.
    :param end: The line ending. Defaults to "\n" but can be set to "" to not have a linebreak.
    """
    if PRINT_INFO:
        time_stamp = __datetime.datetime.fromtimestamp(__time.time()).strftime('%Y-%m-%d %H:%M:%S')
        data = "[{}] INFO {}".format(time_stamp, msg)
        print("\r{}".format(data), end=end)
        if __logfile is not None:
            with open(__logfile, "a") as f:
                f.write(data + "\n")
        else:
            __log_buffer.append(data)

In [22]:
#export
def warn(msg: str, end: str = "\n") -> None:
    """
    Print something with a timestamp.
    Useful for logging.
    Babilim internally uses this for all its log messages.

    .. code-block:: python

        from babilim import tprint
        tprint("This is a log message.")
        # [yyyy-mm-dd HH:MM:SS] This is a log message.

    :param msg: The message to print.
    :param end: The line ending. Defaults to "\n" but can be set to "" to not have a linebreak.
    """
    if PRINT_WARN:
        time_stamp = __datetime.datetime.fromtimestamp(__time.time()).strftime('%Y-%m-%d %H:%M:%S')
        data = "[{}] WARN {}".format(time_stamp, msg)
        print("\r{}".format(data), end=end)
        if __logfile is not None:
            with open(__logfile, "a") as f:
                f.write(data + "\n")
        else:
            __log_buffer.append(data)

In [23]:
#export
def error(msg: str, end: str = "\n") -> None:
    """
    Print something with a timestamp.
    Useful for logging.
    Babilim internally uses this for all its log messages.

    .. code-block:: python

        from babilim import tprint
        tprint("This is a log message.")
        # [yyyy-mm-dd HH:MM:SS] This is a log message.

    :param msg: The message to print.
    :param end: The line ending. Defaults to "\n" but can be set to "" to not have a linebreak.
    """
    if PRINT_ERROR:
        time_stamp = __datetime.datetime.fromtimestamp(__time.time()).strftime('%Y-%m-%d %H:%M:%S')
        data = "[{}] ERROR {}".format(time_stamp, msg)
        print("\r{}".format(data), end=end)
        if __logfile is not None:
            with open(__logfile, "a") as f:
                f.write(data + "\n")
        else:
            __log_buffer.append(data)